In [16]:
import sys
sys.path.append("..")

from pathlib import Path

import torch
from pytorch_lightning import Trainer

from src.intent.data.manager import IntentDataManager
from src.intent.models import IntentClassifier

data_dir = Path("../data/intent")
cache_dir = Path("../cache/intent")

In [8]:
data_manager = IntentDataManager(
    cache_dir=cache_dir,
    max_len=64,
    batch_size=128,
    num_workers=8,
    test_file=data_dir / "test.json"
)

2022-02-26 13:37:02 | INFO | Vocab loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/vocab.pkl
2022-02-26 13:37:02 | INFO | Intent-2-Index loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/intent2idx.json
2022-02-26 13:37:02 | INFO | Embeddings loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/embeddings.pt


In [13]:
test_dataloader = data_manager.get_test_dataloader()
len(test_dataloader)

36

In [10]:
ckpt_path = Path("../ckpt/intent/20220226_2119/intent-epoch=04-val_acc=0.88-val_loss=0.69.ckpt")
model = IntentClassifier.load_from_checkpoint(ckpt_path)

In [11]:
trainer = Trainer(
    devices=[7],
    accelerator="gpu",
    deterministic=True,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [12]:
pred = trainer.predict(model, test_dataloader, return_predictions=True)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting: 100%|██████████| 36/36 [00:01<00:00, 28.07it/s]


In [28]:
# pred = torch.cat(pred)
pred.shape

torch.Size([4500])

In [23]:
import json
ids = [d["id"] for d in json.loads(Path("../data/intent/test.json").read_text())]
len(ids)

4500

In [33]:
result = [
    {"id": i, "intent": p}
    for i, p in zip(ids, pred.tolist())
]
result[:5]

[{'id': 'test-0', 'intent': 67},
 {'id': 'test-1', 'intent': 86},
 {'id': 'test-2', 'intent': 14},
 {'id': 'test-3', 'intent': 18},
 {'id': 'test-4', 'intent': 34}]